# Model for predicting sentiment value of text entered using deep-learning

In [1]:
#for cleaning the text removing everything except the characters(letters and numbers)
def cleanString(incomingString):
    newstring = incomingString
    newstring = newstring.replace("!","")
    newstring = newstring.replace("@","")
    newstring = newstring.replace("#","")
    newstring = newstring.replace("$","")
    newstring = newstring.replace("%","")
    newstring = newstring.replace("^","")
    newstring = newstring.replace("&","and")
    newstring = newstring.replace("*","")
    newstring = newstring.replace("(","")
    newstring = newstring.replace(")","")
    newstring = newstring.replace("+","")
    newstring = newstring.replace("=","")
    newstring = newstring.replace("?","")
    newstring = newstring.replace("\'","")
    newstring = newstring.replace("\"","")
    newstring = newstring.replace("{","")
    newstring = newstring.replace("}","")
    newstring = newstring.replace("[","")
    newstring = newstring.replace("]","")
    newstring = newstring.replace("<","")
    newstring = newstring.replace(">","")
    newstring = newstring.replace("~","")
    newstring = newstring.replace("`","")
    newstring = newstring.replace(":","")
    newstring = newstring.replace(";","")
    newstring = newstring.replace("|","")
    newstring = newstring.replace("\\","")
    newstring = newstring.replace("/","")
    newstring = newstring.replace(",","")
    newstring = newstring.replace(".","")
    return newstring

In [2]:
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras import utils


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
print("Tensorflow version", tf.__version__)

Tensorflow version 1.5.0


#### loading the data

In [4]:
data = pd.read_csv('/Users/vishalsingh/Desktop/Saber/csvfor16k.csv',encoding = 'ISO-8859-1')

In [5]:
from sklearn.utils import shuffle
data = shuffle(data)

In [6]:
data.columns

Index(['Unnamed: 0', 'Author', 'Category', 'Company', 'Content', 'Date',
       'Sentiment', 'Source', 'Subtitle', 'Tags', 'Title', 'Unnamed: 0.1',
       'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1'],
      dtype='object')

#### Split the data into train and test

In [7]:
train_size = int(len(data)*.80)
print('Train size:', train_size)

Train size: 12800


In [8]:
#for training the model
train_posts = data['Title'][:train_size]
train_posts = [ cleanString(x) for x in train_posts] 
train_tags = data['Sentiment'][:train_size]  
train_tags = [round(x,1)  for x in train_tags]

#the model will be trained on 10 labels i.e 0.0 .1 .2 .3 .4 .5 .6 .7 .8 .9 1.0
    
#for testing the trained model....
test_posts = data['Title'][train_size:]
test_posts = [cleanString(x) for x in test_posts]
test_tags = data['Sentiment'][train_size:]
test_tags = [round(x,1)  for x in test_tags]  

for i in test_posts[:100]:
    print (i)

Sistema Shyam Tele-Services prefers bandwidth sharing to improve access
Apcotex Industries AGM on August 10 2016
Apollo Tyres plans 440 million Euro investment in Hungary
Rajasthan may face blackout from Sunday as NTPC threatens to snap supply for defaulting on Rs 367 crore pay
Railways blames Bihar government for train accident
Vibrant Gujarat PM Narendra Modi promises to make India easiest place to do business
Apollo Hospitals family to form trust
Kingfisher Airlines accepts CEO Sanjay Aggarwals resignation
Job losses not a worry as data demand will keep telecom afloat in fiscal storm
Ansal Properties and Infrastructure consider dividend
ANG Industries Receipt of Order
Mamta Banerjee to face Congress Left heat in Delhi
Indias growth to accelerate to 75 this fiscal Moodys
Apollo Tyres jumps 6 on falling input costs
LafargeHolcim increases stake in Ambuja Cements ACC
Apollo Hospitals introduces a new paradigm in Cancer Care
No proposal to set up new commands for space assets Antony
2G 

In [9]:
for i in range(10):
    print(test_tags[i])
#labels for first 10 entries...    

0.6
0.8
1.0
0.3
0.4
0.9
0.9
0.8
0.9
0.8


### tokenize the data

In [10]:
from keras.preprocessing import text

In [11]:
max_words = 3000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [12]:
tokenize.fit_on_texts(train_posts)
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [13]:
len(x_test[40])

3000

### Use sklearn utility to convert label strings to numbered index


In [14]:
encoder = LabelEncoder()
encoder.fit(train_tags)

LabelEncoder()

In [15]:
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [16]:
y_train

array([3, 0, 4, ..., 3, 0, 3])

In [17]:
y_test

array([ 6,  8, 10, ...,  7,  1,  1])

In [18]:
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [19]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (12800, 3000)
x_test shape: (3200, 3000)
y_train shape: (12800, 11)
y_test shape: (3200, 11)


In [20]:
batch_size = 32

In [21]:
### Build the model
model = Sequential()
model.add(Dense(512,input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])


In [22]:
history = model.fit(x_train,y_train,
                   batch_size=batch_size,
                   epochs=10,
                   verbose=1,
                   validation_split=0.1)

Train on 11520 samples, validate on 1280 samples
Epoch 1/10
11520/11520 [==============================] - 6s 532us/step - loss: 2.2319 - acc: 0.1644 - val_loss: 2.0935 - val_acc: 0.2242
Epoch 2/10
11520/11520 [==============================] - 6s 492us/step - loss: 1.9675 - acc: 0.2859 - val_loss: 1.9645 - val_acc: 0.2562
Epoch 3/10
11520/11520 [==============================] - 6s 492us/step - loss: 1.7536 - acc: 0.3758 - val_loss: 1.9172 - val_acc: 0.2680
Epoch 4/10
11520/11520 [==============================] - 6s 491us/step - loss: 1.5736 - acc: 0.4490 - val_loss: 1.9005 - val_acc: 0.2820
Epoch 5/10
11520/11520 [==============================] - 6s 543us/step - loss: 1.4255 - acc: 0.5008 - val_loss: 1.9192 - val_acc: 0.2875
Epoch 6/10
11520/11520 [==============================] - 6s 551us/step - loss: 1.3043 - acc: 0.5464 - val_loss: 1.9372 - val_acc: 0.2961
Epoch 7/10
11520/11520 [==============================] - 7s 612us/step - loss: 1.1980 - acc: 0.5915 - val_loss: 1.9684 - v

In [23]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

#the accuracy score below is meant for 11 labels if ... we train the model on binary data then 
#this model gives a state of art accuracy of 99%

3200/3200 [==============================] - 0s 123us/step
Test score: 2.1175447309017184
Test accuracy: 0.2928125


In [26]:
text_labels = encoder.classes_ 
#output below....
for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_posts[i])
    print('Actual label:' + str(test_tags[i]))
    print("Predicted label: " + str(predicted_label) + "\n")

Sistema Shyam Tele-Services prefers bandwidth sharing to improve access
Actual label:0.6
Predicted label: 0.9

Apcotex Industries AGM on August 10 2016
Actual label:0.8
Predicted label: 0.9

Apollo Tyres plans 440 million Euro investment in Hungary
Actual label:1.0
Predicted label: 0.9

Rajasthan may face blackout from Sunday as NTPC threatens to snap supply for defaulting on Rs 367 crore pay
Actual label:0.3
Predicted label: 0.7

Railways blames Bihar government for train accident
Actual label:0.4
Predicted label: 0.6

Vibrant Gujarat PM Narendra Modi promises to make India easiest place to do business
Actual label:0.9
Predicted label: 0.3

Apollo Hospitals family to form trust
Actual label:0.9
Predicted label: 0.8

Kingfisher Airlines accepts CEO Sanjay Aggarwals resignation
Actual label:0.8
Predicted label: 0.6

Job losses not a worry as data demand will keep telecom afloat in fiscal storm
Actual label:0.9
Predicted label: 0.9

Ansal Properties and Infrastructure consider dividend
A

In [27]:
#saving the model so that we do not need to train the model again and again....
from keras.models import load_model
model.save('model_file.h5')
my_model = load_model('model_file.h5')

In [91]:
my_model